In [15]:
#Credentials for the azure Ner service
key = "3c516844a8274d61b863123f938f9cea"
endpoint = "https://szwb-azure.cognitiveservices.azure.com/"

#Imports
import pandas as pd
from azure.ai.textanalytics import TextAnalyticsClient
from azure.ai.textanalytics import CategorizedEntity
from azure.core.credentials import AzureKeyCredential
import re


fields = ['Sentences','Word', 'POS', 'Tag']

train = pd.read_csv("./data/TrainNER.csv",sep=';',na_values='', encoding='latin1')
train = train.rename(columns={train.columns[0]: 'Sentences', train.columns[1]: 'Word',train.columns[2]:  'POS',train.columns[3]:  'Tag'}) # adding first row
del train["POS"]
train["Predicted"]=""


test1 = pd.read_csv("./data/Test1NER.csv",sep=';',na_values='', encoding='latin1')
test2 = pd.read_csv("./data/Test2NER.csv",sep=';',na_values='', encoding='latin1')

#Add columns to test1 df
test1.loc[-1] = [test1.columns[0], test1.columns[1], test1.columns[2]]  # rename columns
test1.index = test1.index + 1  # shifting index
test1 = test1.sort_index()  # sorting by index
test1 = test1.rename(columns={test1.columns[0]: 'Sentences', test1.columns[1]: 'Word',test1.columns[2]:  'POS'}) # adding first row
test1["Predicted"]=""

#Add columns to test2 df
test2.loc[-1] = [test2.columns[0], test2.columns[1]]  # rename columns
test2.index = test2.index + 1  # shifting index
test2 = test2.sort_index()  # sorting by index
test2 = test2.rename(columns={test2.columns[0]: 'Sentences', test2.columns[1]: 'Word'}) # adding first row
test2["Predicted"]=""

In [16]:
#Extract the sentences one-by-one from the tokenized csv file

def getAllSentences(df):
    all_sentences = list()
    s = ""
    for w in df.index:
        if not pd.notna(df.iloc[w]['Sentences']):
            if (not (('É' in str(df.iloc[w]['Word'])) or ('Ó' in str(df.iloc[w]['Word'])))):
                s = s+str(df.iloc[w]['Word'])+" "
        else:
            s = s[:-1]
            all_sentences.append(s)
            s = ""
            if (not (('É' in str(df.iloc[w]['Word'])) or ('Ó' in str(df.iloc[w]['Word'])))):
                s = s+str(df.iloc[w]['Word'])+" "
    s = s[:-1]
    all_sentences.append(s)
    all_sentences.pop(0)
    return all_sentences    

#getAllSentences(test1)
#print(all_sentences[2])

In [17]:
#The azure NER client initalization and the entity recognition function.

def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint, 
            credential=ta_credential)
    return text_analytics_client

client = authenticate_client()

def entity_recognition(client,documents):
    try:
        result = client.recognize_entities(documents= documents)[0]
        #The response contains compley entites, not word-by-word
        return result

    except Exception as err:
        print("Encountered exception. {}".format(err))

In [18]:
#Adds BIO tagging to the prediction.
def bio_tagger(ne_tagged):
		bio_tagged = []
		prev_tag = "O"
		for token, tag in ne_tagged:
			if tag == "O": #O
				bio_tagged.append((token, tag))
				prev_tag = tag
				continue
			if tag != "O" and prev_tag == "O": # Begin NE
				bio_tagged.append((token, "B-"+tag))
				prev_tag = tag
			elif prev_tag != "O" and prev_tag == tag: # Inside NE
				bio_tagged.append((token, "I-"+tag))
				prev_tag = tag
			elif prev_tag != "O" and prev_tag != tag: # Adjacent NE
				bio_tagged.append((token, "B-"+tag))
				prev_tag = tag
		return bio_tagged

#Calls the Azure Ner interface with the given test data
#Returns the results in a list of tuples, what contains the (word,prediction) pairs
def get_prediction(df):
    allEntity = list()

    submit = data_preparation(df)
    
    for i in range(len(submit)):

        res = entity_recognition(client,submit[i:i+1])
        l = len(res.entities)
        filtered_res = list()
        
        for index, entity in enumerate(res.entities):
            if index == 0:
                filtered_res.append(entity)
            if index > 0:
                previous =  res.entities[index - 1]
                if not  entity.text in previous.text:
                    filtered_res.append(entity)
        for entity in filtered_res:
            if(len(entity.text.split(" "))>1):
                s=entity.text.split(" ")
                for t in s:
                    if t[0] == ".":
                        t = t.replace(".","")
                    tup = (t,str(entity.category)+'-'+str(entity.subcategory))
                    allEntity.append(tup)
            else:
                if entity.text[0] == ".":
                        entity.text = entity.text.replace(".","",1)
                tup = (entity.text,str(entity.category)+'-'+str(entity.subcategory))
                allEntity.append(tup)
    formated_entity = list()
    l = len(allEntity)
    for index, entity in enumerate(allEntity):
            if index == (l - 1):
                formated_entity.append(entity)
            if index < (l - 1):
                next = allEntity[index + 1]
                if not (entity[0] == next[0] and entity[1] == entity[1]):
                    formated_entity.append(entity)
    category_filtered_entity = list()
    for w in formated_entity:
        if "Event" in w[1]:
            category_filtered_entity.append(w)
        if "Organization" in w[1]:
            category_filtered_entity.append(w)
        if "Location" in w[1]:
            category_filtered_entity.append(w)
        if "Person" in w[1]:
            category_filtered_entity.append(w)
        if "Time" in w[1]:
            category_filtered_entity.append(w)

    shite_filtered_entity = list()
    l = len(category_filtered_entity)
    for index, entity in enumerate(category_filtered_entity):
            if index == (l - 1):
                shite_filtered_entity.append(entity)
            if index < (l - 1):
                next = category_filtered_entity[index + 1]
                if index < (l - 2):
                    next2 = category_filtered_entity[index + 2]
                
                if 'provinceOfficials' in entity[0]:
                    temp1 = ('province',entity[1])
                    temp2 = ('Officials',entity[1])
                    shite_filtered_entity.append(temp1)
                    shite_filtered_entity.append(temp2)
                if 'BaseThe' in entity[0]:
                    temp1 = ('Base',entity[1])
                    temp2 = ('The',entity[1])
                    shite_filtered_entity.append(temp1)
                    shite_filtered_entity.append(temp2)
                elif 'Shi' in entity[0] and 'ite' in next[0]:
                    temp = ("Shi'ite",entity[1])
                    shite_filtered_entity.append(temp)
                elif (not ('ite' in entity[0])) and (not ('Independence' in entity[0] and 'holiday' in next2[0])) and (not ('Day' in entity[0] and 'holiday' in next[0])):
                    shite_filtered_entity.append(entity)
    return shite_filtered_entity

#Segments up the list what contains the sentences
#This is necessary, beacuse the azure NER has a char size limitation of 5120
#The Azure NER has also a limitation of max 5 docs/request, and a 1MB request size.
def data_preparation(df):
    submit = list()
    allSentences = getAllSentences(df)
    temp=""
    for i in range(len(allSentences)):
        temp = temp + allSentences[i]
        temp = temp.replace("\\",'')
        if(len(temp) > 4750):
            temp = temp.replace("\\",'')
            submit.append(temp)
            temp=""
    submit.append(temp)
    return submit

#print(getPred)

In [19]:
#Regexes for the prediction filter, to shape them into the expected format
b_event_regex = re.compile('(B-Event)+')
i_event_regex = re.compile('(I-Event)+')
b_org_regex = re.compile('(B-Organization)+')
i_org_regex = re.compile('(I-Organization)+')
b_person_regex = re.compile('(B-Person)+')
i_person_regex = re.compile('(I-Person)+')
b_time_regex = re.compile('B-.*(Time)+')
i_time_regex = re.compile('I-.*(Time)+')

#Insert the result of prediction into the given dataframe
def insertPred(df,result):
    i=0
    for index, row in df.iterrows():  
        if result[i][0] in row['Word']:
            row['Predicted']=result[i][1]
            if i < len(result)-1:
                i = i+1
        else:
            row['Predicted']='O'

def prediction_formatting(df):
    for w in df.index:
        if df['Predicted'][w]=='B-Location-GPE':
            df['Predicted'][w]='B-gpe'
        elif df['Predicted'][w]=='I-Location-GPE':
            df['Predicted'][w]='I-gpe'

        elif re.match(b_event_regex, df['Predicted'][w]):
            df['Predicted'][w]='B-event'
        elif re.match(i_event_regex, df['Predicted'][w]):
            df['Predicted'][w]='I-event'

        elif df['Predicted'][w]=='B-Location-Geographical':
            df['Predicted'][w]='B-geo'
        elif df['Predicted'][w]=='I-Location-Geographical':
            df['Predicted'][w]='I-geo'

        elif df['Predicted'][w]=='B-Location-Structural':
            df['Predicted'][w]='B-obj'
        elif df['Predicted'][w]=='I-Location-Structural':
            df['Predicted'][w]='I-obj'

        elif re.match(b_org_regex, df['Predicted'][w]):
            df['Predicted'][w]='B-org'
        elif re.match(i_org_regex, df['Predicted'][w]):
            df['Predicted'][w]='I-org'

        elif re.match(b_person_regex, df['Predicted'][w]):
            df['Predicted'][w]='B-per'
        elif re.match(i_person_regex, df['Predicted'][w]):
            df['Predicted'][w]='I-per'

        elif re.match(b_time_regex, df['Predicted'][w]):
            df['Predicted'][w]='B-time'
        elif re.match(i_time_regex, df['Predicted'][w]):
            df['Predicted'][w]='I-time'
        else:
            df['Predicted'][w]='O'



#Tagging all words
def tag_all_words(df):
    ne_tags = df["Predicted"]
    tokens =  df["Word"]
    tups = list()
    for f,s in zip(ne_tags,tokens):
        tups.append((s,f))
    tokens,ne_tags = zip(*bio_tagger(tups))
    df["Predicted"] = ne_tags




In [12]:
#Command sequence
insertPred(test1,get_prediction(test1))
tag_all_words(test1)
prediction_formatting(test1)

In [20]:
insertPred(test2,get_prediction(test2))
tag_all_words(test2)
prediction_formatting(test2)

In [13]:
test1.to_csv("Submission1_Azure.csv",index=False,sep = ';',encoding='utf-8')


In [26]:
test2.to_csv("Submission2_Azure.csv",index=False,sep = ';',encoding='ANSI')